In [1]:
import tensorflow as tf
import numpy as np

In [20]:
class MultiheadAttention(tf.keras.layers.Layer):
    def __init__(self, num_heads, d_model):
        super().__init__()
        self.d_model = d_model  # e.g., 300
        self.num_heads = num_heads  # e.g., 10

        self.depth = d_model // self.num_heads  # e.g., 30

        self.wq = tf.keras.layers.Dense(d_model)
        self.wk = tf.keras.layers.Dense(d_model)
        self.wv = tf.keras.layers.Dense(d_model)
        
        self.dense = tf.keras.layers.Dense(d_model)

    def split_heads(self, x, batch_size):
        # Shape of x: (batch_size, n, d_model)
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])
    
    def scaled_dot_product_attention(self, q, k, v, mask, causal_mask):
        # Shape of q, k, v: (batch, num_heads, n, depth)
        n = q.shape[2]
        matmul_qk = tf.matmul(q, k, transpose_b=True)  # Shape: (batch, num_heads, n, d) * (batch, num_heads, d, n) --> (batch, num_heads, n, n)

        d_k = tf.cast(tf.shape(k)[-1], tf.float32)
        scaled_attention_logits = matmul_qk / tf.math.sqrt(d_k)  # Shape: (batch, num_heads, n, n)

        if mask is not None:
            # Originally mask is of shape (batch, n), but here:
            # Mask should be of shape: (batch, 1, 1, n)
            scaled_attention_logits += (mask * -1e9)  # Apply mask

        if causal_mask:
            mask = tf.linalg.band_part(tf.ones((n, n), dtype=tf.int32), num_lower=-1, num_upper=0)  # Shape: (n, n)
            mask = tf.where(mask==0, 1e-9, 1)
            scaled_attention_logits = scaled_attention_logits * mask

        attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # Shape: (batch, num_heads, n, n)

        output = tf.matmul(attention_weights, v)  # Shape: (batch, num_heads, n, depth)

        return output, attention_weights

    
    def call(self, query, key, value, mask, causal_mask):
        # Shape of x: (batch_size, n, d_model)
        batch_size = tf.shape(query)[0]

        q = self.wq(query)
        k = self.wk(key)
        v = self.wv(value)

        q = self.split_heads(q, batch_size)
        k = self.split_heads(k, batch_size)
        v = self.split_heads(k, batch_size)

        scaled_attention, attention_weights = self.scaled_dot_product_attention(q, k, v, mask, causal_mask)
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.d_model))

        return concat_attention, attention_weights